In [84]:
%reload_ext autoreload
%autoreload 2

from pprint import pprint
import csv
from copy import deepcopy
import json
import pathlib
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
import src.annotation_utils as a_utils
import src.llm_utils as llm_utils
import src.message_utils as m_utils
import os

load_dotenv()

BRAT_DATA_PATH = os.environ['BRAT_DATA_PATH']
DATA_DEF_FILE = os.environ['DATA_DEF_FILE']


## Load brat dataset

In [27]:

import dataclasses

from pybrat.parser import BratParser, Entity, Event, Example, Relation

# Initialize a parser.
brat = BratParser(error="ignore")
# brat = BratParser()
annotations = brat.parse(BRAT_DATA_PATH)


In [72]:
# data_entities = a_utils.get_data_entities_of_segments(annotations, DATA_DEF_FILE)
data_entities = a_utils.get_data_entities_of_sentences(annotations, DATA_DEF_FILE)

len(data_entities), data_entities[10]

(1067,
 {'segment': 'INTRODUCTION\n\nDomains Holdings Group Limited (the "Company"), offers a global network of content-rich web, mobile and social sites, including user acquisition and media buying (including paid search, display, mobile and social advertising), content optimization (including key word search, off-page and on-page optimization), and affiliate program, both in the web and mobile ecosystem (including through websites or mobile apps operated by Company; collectively, the "Services").\nThe Company is deeply committed to safeguard the privacy expectations of its end users ("User(s)", "you" or "your"). Accordingly, we have put in place this Privacy Policy, which outlines our data protection practices, including how we collect, use, disclose and protect your Personal Information, as well as your rights with respect to your Personal Information.\nIn this Privacy Policy, you can learn about:\nWHAT INFORMATION WE COLLECT?\nMINORS\nHOW WE USE THE COLLECTED INFORMATION?\nMARKETIN

## Select training data

In [73]:
def has_no_entities(segment):
    return not segment["entities"]

empty_entity_indices = [i for i, segment in enumerate(data_entities) if has_no_entities(segment)]

training_set_indices = []
validation_set_indices = []
for i, segment in enumerate(data_entities):
    if not has_no_entities(segment):
        if len(training_set_indices) < 20:
            training_set_indices.append(i)
        elif len(validation_set_indices) < 3:
            validation_set_indices.append(i)

for i in empty_entity_indices:
    if len(training_set_indices) < 30:
        training_set_indices.append(i)
    elif len(validation_set_indices) < 5:
        validation_set_indices.append(i)

len(training_set_indices), len(validation_set_indices)

(30, 5)

## Fine-tune model with specified training data

In [76]:
# all_data = m_utils.as_training_data_for_data_span_of_segment(data_entities)
all_data = m_utils.as_training_data_for_data_span_of_sentence(data_entities)
# all_data = m_utils.as_training_data_for_data_classification_of_segment(data_entities)

base_model = 'gpt-4o-mini-2024-07-18'
suffix = 'seg-sent-data-entities-30_train-5_val-with_empty'

In [78]:
job_desc_dir, fine_tune_job, test_set = llm_utils.fine_tune_with_data(
    all_data, training_set_indices, validation_set_indices,
    BRAT_DATA_PATH, DATA_DEF_FILE,
    basemodel=base_model,
    fine_tune_args={'suffix':suffix},
    desc='')

job_desc_dir, fine_tune_job, len(test_set), test_set[0]

Constructing training and validation data files (locally)...
Uploading training and validation data files...
Data files uploading (API calls returned).
Creating fine-tuning job...


InternalServerError: Error code: 500 - {'error': {'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_1ba4e940c152a7b11114adae3f298f86 in your email.)', 'type': 'server_error', 'param': None, 'code': None}}

In [41]:
llm_utils.await_fine_tune_finish_and_clean_up(wait_for_job_completion=False)

FineTuningJob(id='ftjob-61c0FVgYkOfjZaMwRpOjNOzo', created_at=1726265855, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-2024-08-06:rui:30-train-5-val-with-empty-from-api:A790JXld', finished_at=1726266269, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-B2C2pNzAq4paAOvhIYdFJlSv', result_files=['file-1TTTIJ6t7DVXg51yYafkwd4J'], seed=1504396448, status='succeeded', trained_tokens=68019, training_file='file-iYtsamFpHaR3FyqGhHWNeMVm', validation_file='file-C1S6NC5e9VWZrwgmuFzkDxVR', estimated_finish=None, integrations=[], user_provided_suffix='30_train-5_val-with_empty-from_api')

In [31]:
fine_tuned_model_id = 'ft:gpt-4o-mini-2024-07-18:rui:30_train-5_val-content_only-from_api:A6KhgqGM'
fine_tuned_model_id = 'ft:gpt-4o-mini-2024-07-18:rui:30-train-5-val-content-only-from-api:A6cig7w6'